In [ ]:
#| default_exp augmentations

# Augmentations

> Because this will help

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np, torch, torch.nn.functional as F
from lightning.pytorch.callbacks import Callback
from torch.distributions.beta import Beta


## Patching

In [ ]:
#| export 
def create_patch(xb, patch_len, stride, return_patch_num=False, constant_pad=False, constant_pad_value=0, max_seq_len=None):
    """
    xb: [bs x n_vars x seq_len]
    """
    seq_len = xb.shape[-1] if max_seq_len is None else max_seq_len
    xb = xb[...,:seq_len]
    patch_num = (max(seq_len, patch_len)-patch_len) // stride + 1  # in supervised tsai model - patch_num = int((seq_len - patch_len) / stride + 1) + 1
    patch_len = seq_len if patch_len > seq_len else patch_len
    if constant_pad:
        # seq_len > patch_len and stride <= seq_len - patch_len
        if ((seq_len-patch_len) % stride != 0):
            # only pad if remainder
            xb = F.pad(xb, (0, stride), 'constant', value=constant_pad_value) # pad at the end with value
            patch_num += 1
    dim = xb.dim()
    dimension = 0
    if dim > 3:
        raise ValueError('Tensor has dimension gt 3, not sure how to patch.')
    elif dim == 3:
        xb = xb.permute(0,2,1) # xb: [bs x tgt_len x nvars]
        dimension = 1
    elif dim == 2:
        xb = xb.permute(1,0)
    xb = xb.unfold(dimension=dimension, size=patch_len, step=stride) # xb: [bs x num_patch x n_vars x patch_len]
    if dim == 2:
        xb = xb.transpose(0,1) #[bx x num_patch x patch_len]]
    if return_patch_num:
        return xb, patch_num
    else:
        return xb

In [ ]:
#| notest
x = torch.randn(4, 1000)

# test seq_len > patch len == stride 
xb = create_patch(x, patch_len=505, stride=500, constant_pad=False)
xb_rep = create_patch(x, patch_len=500, stride=500, constant_pad=True)
x.shape, xb.shape, xb_rep.shape
#xb_rep_short = create_patch(x_short, patch_en=502, stride=500, replication_pad=False)b

(torch.Size([4, 1000]), torch.Size([4, 1, 505]), torch.Size([4, 2, 500]))

In [ ]:
#| notest
x = torch.randn(1,7,1350000)

# test seq_len > patch len == stride 
xb = create_patch(x, patch_len=1024, stride=1024, constant_pad=False)
xb_rep = create_patch(x, patch_len=1024, stride=1024, constant_pad=True)
x.shape, xb.shape, xb_rep.shape
#xb_rep_short = create_patch(x_short, patch_en=502, stride=500, replication_pad=False)

(torch.Size([1, 7, 1350000]),
 torch.Size([1, 1318, 7, 1024]),
 torch.Size([1, 1319, 7, 1024]))

In [ ]:
#| export
def unpatch(x, seq_len, remove_padding=True):
    """
    x: [bs/None x patch_num x n_vars x patch_len]
    returns x: [bs x n_vars x seq_len]
    """
    if x.dim() == 3:
        x = x.transpose(0,1)
    else:
        x = x.transpose(1,2)
    x = x.flatten(start_dim=-2, end_dim=-1)
    if remove_padding:
        x = x[...,:seq_len]
    return x

In [ ]:
#| notest
x = torch.randn(1,1,50)

# test seq_len > patch len == stride 
xb = create_patch(x, patch_len=6, stride=6, constant_pad=True)
xb = unpatch(xb, seq_len=50, remove_padding=False)
xb.shape

(torch.Size([1, 1, 50]), torch.Size([1, 9, 1, 6]))

## Patch Masking

In [ ]:
#| export
def mask_patches_simple(xb, # input tensor of size 3 or 4 to be masked
                        mask_ratio # ratio of masking of patches
                        ):
    """
    Function that masks patches in a simple way

    xb: [bs x patch_num x n_vars x patch_len]
    padding_mask [bs x patch_num x 1|num_vars x patch_len]
    """
    if xb.dim() == 3:
        xb = xb.unsqueeze(0)
    assert xb.dim() == 4, "The simple masking function expects a dim == 4 input tensor with [bs x patch_num x n_vars x patch_len]"
    x = xb.clone()
    mask_to_multiply = torch.from_numpy(np.random.choice([0,1], size=x.shape[:3], p=[mask_ratio,1-mask_ratio])).to(xb.device)
    x = x * mask_to_multiply[...,None]
    mask = (mask_to_multiply == 0).int()
    return x,mask

def random_masking(xb, mask_ratio):
    # xb: [bs x num_patch x n_vars x patch_len]
    bs, L, nvars, D = xb.shape
    x = xb.clone()
    
    len_keep = int(L * (1 - mask_ratio))
        
    noise = torch.rand(bs, L, nvars,device=xb.device)  # noise in [0, 1], bs x L x nvars
        
    # sort noise for each sample
    ids_shuffle = torch.argsort(noise, dim=1)  # ascend: small is keep, large is remove
    ids_restore = torch.argsort(ids_shuffle, dim=1)                                     # ids_restore: [bs x L x nvars]

    # keep the first subset
    ids_keep = ids_shuffle[:, :len_keep, :]                                              # ids_keep: [bs x len_keep x nvars]         
    x_kept = torch.gather(x, dim=1, index=ids_keep.unsqueeze(-1).repeat(1, 1, 1, D))     # x_kept: [bs x len_keep x nvars  x patch_len]
   
    # removed x
    x_removed = torch.zeros(bs, L-len_keep, nvars, D, device=xb.device)                 # x_removed: [bs x (L-len_keep) x nvars x patch_len]
    x_ = torch.cat([x_kept, x_removed], dim=1)                                          # x_: [bs x L x nvars x patch_len]

    # combine the kept part and the removed one
    x_masked = torch.gather(x_, dim=1, index=ids_restore.unsqueeze(-1).repeat(1,1,1,D)) # x_masked: [bs x num_patch x nvars x patch_len]

    # generate the binary mask: 0 is keep, 1 is remove
    mask = torch.ones([bs, L, nvars], device=x.device)                                  # mask: [bs x num_patch x nvars]
    mask[:, :len_keep, :] = 0
    # unshuffle to get the binary mask
    mask = torch.gather(mask, dim=1, index=ids_restore)                                  # [bs x num_patch x nvars]
    return x_masked, mask

In [ ]:
#| notest
x = torch.randn(50,16,7,50)
mask_ratio = 0.4

x_new, mask = mask_patches_simple(x,mask_ratio=mask_ratio)

## Value Augmentations

In [ ]:
#| export
def remove_values(x, mask_ratio):
    mask = torch.rand(x.shape, device=x.device) > mask_ratio # maskout_ratio are False, note that this is per channel masking
    x = x*mask
    return x

def jitter_augmentation(x, mask_ratio=0.05, jitter_ratio=0.05):
    x = x.clone()  
    max_amplitude = x.abs().max()  # Use absolute max for amplitude scale

    zero_mask = torch.rand(x.shape, device=x.device) > mask_ratio
    jitter_mask = torch.rand(x.shape, device=x.device) > (1-jitter_ratio)
    jitter_values = torch.randn(x.shape, device=x.device) * max_amplitude * 0.1  # Normal distribution centered at 0 times 10% of max amplitude
    
    x = x*zero_mask + jitter_mask*jitter_values
    n_masks =  (~zero_mask).sum() + jitter_mask.sum()
    return x, n_masks

In [ ]:
#| notest
## note that the random number generator advances state...
torch.manual_seed(42)
x = torch.randn(4,7,1000)

torch.manual_seed(42)
x_new, n_masks = jitter_augmentation(x)
n_masks /(4* 7*1000)

torch.manual_seed(42)
x_new2, n_masks2 = jitter_augmentation(x)
torch.equal(x_new, x_new2)

True

## Shuffle Augmentations

In [ ]:
#| export
def shuffle_dim(x, dim=1, p=0.5):
    """
    shuffles a dimension randomly along dim
    x: [bs x n channels x n patches x patch len]
    """
    if torch.rand(1, device=x.device) > (1-p):
        idx = torch.randperm(x.size(dim), device=x.device)
        return torch.index_select(x, dim, idx)
    else:
        return x


In [ ]:
#| export
def reverse_sequence(x, seq_dim=(-1,), p=0.5):
    if torch.rand(1, device=x.device) > (1-p):
        return torch.flip(x, dims=seq_dim)
    else:
        return x

In [ ]:
#| notest
x = torch.randn(4,1,5,5).to('cuda')

torch.equal(shuffle_dim(x), x)

False

## Callbacks

In [ ]:
#| export
class IntraClassCutMix1d(Callback):
    """
    Intra-class CutMix for 1D data (e.g., time-series). 

    This is a callback that can be used to apply CutMix to the training data.
    It is used to mix segments within the same class.
    """
    def __init__(self, 
                 mix_prob=0.5, # probability of applying cutmix
                 return_y_every_sec=30, # length of segment to mix, if one value of y corresponds to 30 seconds of signal data, this should be set to 30. 
                 frequency=125, # frequency of the data
                 return_sequence_padding_mask=True # whether to return the sequence padding mask
                 ):
        super().__init__()
        self.mix_prob = mix_prob # probability of applying cutmix
        self.segment_length = return_y_every_sec # length of segment to mix
        self.frequency = frequency # frequency of the data
        self.return_sequence_padding_mask = return_sequence_padding_mask # whether to return the sequence padding mask

    def on_train_batch_start(self, trainer, pl_module, batch, batch_idx):
        if self.return_sequence_padding_mask:
            x, y, padding_mask = batch  # x: [batch_size, channels, time_steps], y: [batch_size, time_steps]
        else:
            x, y = batch  # x: [batch_size, channels, time_steps], y: [batch_size, time_steps]

        if torch.rand(1) < self.mix_prob:
            # Group indices by class
            #class_indices = {cls.item(): (y == cls).nonzero(as_tuple=True) for cls in y.unique()}

            # Number of segments
            n_segments = y.shape[-1]

            for seg in range(n_segments):
                start = seg
                end = seg + 1

                start_x = start*self.segment_length*self.frequency
                end_x = end*self.segment_length*self.frequency
                # Get the classes for this segment across all batch items
                seg_classes = y[:, start].cpu()  # Assuming the class is consistent within a segment

                for cls in seg_classes.unique():
                    # Find batch items with this class in the current segment
                    batch_indices = (seg_classes == cls).nonzero().squeeze(-1)
                    if len(batch_indices) > 1:  # We need at least 2 samples to mix
                        # Shuffle these indices
                        shuffled_indices = batch_indices[torch.randperm(len(batch_indices))]
                        
                        # Mix the segments
                        x[batch_indices, :, start_x:end_x] = x[shuffled_indices, :, start_x:end_x]
       
        return (x, y) if not self.return_sequence_padding_mask else (x, y, padding_mask)

In [ ]:
#| notest
x = torch.randn(4,7,90)
x_c = x.clone()
y = torch.randint(0, 5, size=(4,90//30))
xxt = IntraClassCutMix1d(mix_prob=1, frequency=1, return_y_every_sec=30, return_sequence_padding_mask=False)
batch = (x,y)
xxt.on_train_batch_start(None, None, batch, 0)
torch.equal(x_c, batch[0]) == False

True

In [ ]:
#| export
class IntraClassCutMixBatch(Callback):
    """
    Intra-class CutMix for 1D data (e.g., time-series). 

    This is a callback that can be used to apply CutMix to the training data.
    It is used to mix segments within the same class.

    This is different to IntraClassCutMix1d in that it mixes segments of the same class across batches of data, rather than just at the same segment
    """
    def __init__(self, 
                 mix_prob=0.5, # probability of applying cutmix
                 return_y_every_sec=30, # length of segment to mix, if one value of y corresponds to 30 seconds of signal data, this should be set to 30. 
                 frequency=125, # frequency of the data
                 return_sequence_padding_mask=True, # whether to return the sequence padding mask
                 intra_class_only=True # whether to mix only within same class (True) or across all classes (False)
                 ):
        super().__init__()
        self.mix_prob = mix_prob # probability of applying cutmix
        self.frequency = frequency # frequency of the data
        self.segment_length = return_y_every_sec*self.frequency # length of segment to mix
        self.return_sequence_padding_mask = return_sequence_padding_mask # whether to return the sequence padding mask
        self.intra_class_only = intra_class_only # whether to mix only within same class (True) or across all classes (False)
    
    def on_train_batch_start(self, trainer, pl_module, batch, batch_idx):
        if self.return_sequence_padding_mask:
            x, y, padding_mask = batch  # x: [batch_size, channels, time_steps], y: [batch_size, time_steps]
        else:
            x, y = batch  # x: [batch_size, channels, time_steps], y: [batch_size, time_steps]
        seq_len = x.shape[-1]

        if batch_idx == 0:
            mode = "intra-class" if self.intra_class_only else "cross-class"
            print(f"{mode} CutMixBatch is being applied!")

        if torch.rand(1) < self.mix_prob:
            # Group indices by class
            #class_indices = {cls.item(): (y == cls).nonzero(as_tuple=True) for cls in y.unique()}

            # create patches of length segment_length, [bs x n_patches x n_vars x patch_len]
            x = create_patch(x, patch_len=self.segment_length, stride=self.segment_length, return_patch_num=False, constant_pad=False, constant_pad_value=0)
            if self.intra_class_only:
                unique_classes = y.unique()
                for cls in unique_classes:
                    class_indices = (y == cls).nonzero()
                    if len(class_indices) > 1:
                        # shuffle the indices
                        shuffled_indices = class_indices[torch.randperm(len(class_indices))]
                        # Extract the batch and patch indices
                        batch_indices = class_indices[:, 0]
                        patch_indices = class_indices[:, 1]
                        # mix the segments
                        x[batch_indices, patch_indices] = x[shuffled_indices[:, 0], shuffled_indices[:, 1]]
            else:
                # Cross-class mixing logic
                all_indices = torch.nonzero(torch.ones_like(y))  # Get all valid indices
                if len(all_indices) > 1:
                    shuffled_indices = all_indices[torch.randperm(len(all_indices))]
                    batch_indices = all_indices[:, 0]
                    patch_indices = all_indices[:, 1]
                    x[batch_indices, patch_indices] = x[shuffled_indices[:, 0], shuffled_indices[:, 1]]
                    y[batch_indices, patch_indices] = y[shuffled_indices[:, 0], shuffled_indices[:, 1]]
            # unpatch the data
            x = unpatch(x, seq_len)
        return (x, y) if not self.return_sequence_padding_mask else (x, y, padding_mask)

In [ ]:
#| notest
x = torch.randn(4,7,90)
x_c = x.clone()
y = torch.randint(0, 5, size=(4,90//30))
xxt = IntraClassCutMixBatch(mix_prob=1, frequency=1, return_y_every_sec=30, return_sequence_padding_mask=False)
batch = (x,y)
batch = xxt.on_train_batch_start(None, None, batch, 0)
torch.equal(x_c, batch[0]) == False

intra-class CutMixBatch is being applied!


True

In [ ]:
#| export
class MixupCallback(Callback):
    """
    Mixup for 1D data (e.g., time-series).

    This callback applies Mixup to the training data, blending both the input data and the labels.

    See tsai implementation here: https://github.com/timeseriesAI/tsai/blob/bdff96cc8c4c8ea55bc20d7cffd6a72e402f4cb2/tsai/data/mixed_augmentation.py#L43

    Note that this creates non-integer labels/soft labels. Loss functions should be able to handle this.
    """
    def __init__(self, 
                 num_classes,
                 mixup_alpha=0.4, # alpha parameter for the beta distribution
                 return_sequence_padding_mask=True, # whether to return the sequence padding mask
                 ignore_index=-100 # ignore index
                 ):
        super().__init__()
        self.distrib = Beta(mixup_alpha, mixup_alpha)
        self.mixup_alpha = mixup_alpha
        self.return_sequence_padding_mask = return_sequence_padding_mask
        self.ignore_index = ignore_index
        self.num_classes = num_classes

    def on_train_batch_start(self, trainer, pl_module, batch, batch_idx):
        if self.return_sequence_padding_mask:
            x, y, padding_mask = batch  # x: [batch_size, channels, time_steps], y: [batch_size, time_steps]
        else:
            x, y = batch  # x: [batch_size, channels, time_steps], y: [batch_size, time_steps]

        # Sample lambda from a beta distribution
        if batch_idx == 0:
            print("Mixup is being applied!")
        lam = self.distrib.sample((x.size(0), )).to(x.device) # [bs]
        # our mixing coefficient is always ≥ 0.5
        lam = torch.max(lam, 1 - lam)
        lam = lam.view(-1, 1, 1)  # for input shape [bs, channels, seq_len]

        # Shuffle the batch
        indices = torch.randperm(x.size(0), device=x.device)
        x_shuffled = x[indices]
        y_shuffled = y[indices]

        # create ignore masks
        ignore_mask = (y == self.ignore_index)
        ignore_mask_shuffled = (y_shuffled == self.ignore_index)
        combined_ignore_mask = torch.logical_or(ignore_mask, ignore_mask_shuffled)

        y_clean = torch.where(ignore_mask, torch.zeros_like(y), y)
        y_shuffled_clean = torch.where(ignore_mask_shuffled, torch.zeros_like(y_shuffled), y_shuffled)
        # Create one-hot encodings
        y_onehot = F.one_hot(y_clean.long(), num_classes=self.num_classes).float()
        y_shuffled_onehot = F.one_hot(y_shuffled_clean.long(), num_classes=self.num_classes).float()
        # Zero out the one-hot vectors for ignored indices
        y_onehot = torch.where(ignore_mask.unsqueeze(-1), torch.zeros_like(y_onehot), y_onehot)
        y_shuffled_onehot = torch.where(ignore_mask_shuffled.unsqueeze(-1), torch.zeros_like(y_shuffled_onehot), y_shuffled_onehot)

        # Mixup the inputs and labels
        x_mixed = torch.lerp(x_shuffled, x, lam) # x = lam * x + (1 - lam) * x_shuffled
        y_mixed = torch.lerp(y_shuffled_onehot, y_onehot, lam) # y = lam * y + (1 - lam) * y_shuffled

        # finally assign all probabilities with a -100 mixing to 0
        y_mixed = torch.where(combined_ignore_mask.unsqueeze(-1), torch.zeros_like(y_mixed), y_mixed)
        # # add back in the ignore index where it was mixed with other labels
        # combined_ignore_mask = torch.logical_or(ignore_mask, ignore_mask_shuffled)
        # if combined_ignore_mask.any():
        #     # y_mixed = torch.where(combined_ignore_mask, 
        #     #                         torch.tensor(self.ignore_index, dtype=y_mixed.dtype, device=y_mixed.device), 
        #     #                     y_mixed)
        #     y_mixed = torch.where(
        #         combined_ignore_mask.unsqueeze(-1),
        #         torch.zeros_like(y_mixed),
        #         y_mixed
        #     )
        y_mixed = y_mixed.permute(0,2,1) # [bs x n_classes x n_patches]
        if self.return_sequence_padding_mask:
            # Mix the padding masks using OR operation (1 means padded)
            padding_mask_shuffled = padding_mask[indices]
            # If either mask indicates padding (1), the result should be padded
            padding_mask = torch.logical_or(padding_mask, padding_mask_shuffled)
            batch[0], batch[1], batch[2] = x_mixed, y_mixed, padding_mask
            #return (x_mixed, y_mixed, padding_mask)
        else:
            batch[0], batch[1] = x_mixed, y_mixed
            #return (x_mixed, y_mixed)
        #return batch

In [ ]:
#| notest
x = torch.randn(4,7,90)
x_c = x.clone()
y_og = torch.randint(0, 5, size=(4,90//30))
y_og[1,2] = -100
y_og[2,1] = -100
y_c = y_og.clone()
xxt = MixupCallback(num_classes=5, mixup_alpha=0.4, return_sequence_padding_mask=False)
batch = (x,y_og)
batch = xxt.on_train_batch_start(None, None, batch, 0)
torch.equal(x_c, batch[0]) == False, torch.equal(y_c, batch[1]) == False

Mixup is being applied!


(True, True)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()